In [81]:
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
from torchvision import transforms

In [82]:
loaded = np.load('data/in10_split_converted.npz', allow_pickle=True)
x_train = loaded['x_train']
y_train = loaded['y_train']
x_test = loaded['x_test_none']
y_test = loaded['y_test']

In [83]:
N, D_in, H1, H2, H3, D_out = x_train.shape[0], x_train.shape[1], 2000, 1000, 500, 10
learning_rate = 0.001

x = torch.Tensor(x_train)
y = np.squeeze(torch.LongTensor(y_train))

In [86]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.Sigmoid(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(H1, H2),
    torch.nn.Sigmoid(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(H2, H3),
    torch.nn.ReLU(),
    torch.nn.Linear(H3, D_out),
    torch.nn.LogSoftmax(dim = 1),
).cuda()

loss_fn = torch.nn.NLLLoss()
    
losses = []

batchs = 256
    
trainset = torch.utils.data.TensorDataset(x, y)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batchs,
                                          shuffle=True, num_workers=1)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

batchesn = int(N/batchs)
epochs = 25

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, datap in enumerate(trainloader, 0):
        #print(i)
        # get the inputs
        inputs, labels = datap
        inputs = inputs.to('cuda', non_blocking=True)
        labels = labels.to('cuda', non_blocking=True)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % batchesn == batchesn-1:
            print('[epoch {}] loss: {}'.format(epoch + 1, running_loss/batchesn))
            running_loss = 0.0
    scheduler.step()